In [1]:
import time
import random

def random_k_sat(n,m,k):
    # n variables, m clauses, k literals per clause
    clauses = []
    for _ in range(m):
        ret = []
        for _ in range(k):
            v = random.randrange(n)
            polarity = random.randint(0, 1)
            ret.append((polarity, v)) 
        clauses.append(ret)
    vars = [ f"x{idx}" for idx in range(n)]
    return vars, "&".join(
        "(" + "|".join(
            vars[idx] if pol==1 else "~" + vars[idx] for 
            (pol, idx) in cl
        ) + ")" for cl in clauses) 

def time_op(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        val = func(*args, **kwargs)
        dt_s = (time.time() - start)
        print(f"[{dt_s*1000:010.4f} ms / {dt_s:04.4f} s / {dt_s/60:02.4f} min] {func.__name__}") # {args}")
        return val
    return wrapper

vars, formula = random_k_sat(5,5,4)

In [2]:
import importance_measures.bdds as bdd 
with bdd.Buddy(vars) as model:
    x = "x0"
    f = model.node(formula)
    f0 = f & model.node(x) # .restrict(~model.node(x))
    print(f0.satcount) # this is wrong!!!
    print(f"bdd size = {f.nodecount} with satcount = {f.satcount}")
    # f.dump("f.pdf")
    blame, ub = time_op(bdd.blame)(f, x, rho=lambda x: 2**-x, cutoff=0.01, debug=True)
    infl = time_op(bdd.influence)(f, x)
    g = time_op(bdd.omega)(f)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

16.0
bdd size = 3 with satcount = 28.0
=== COMPUTING BLAME for x0 in BDD with size 3 ===
k 0 size g 2 size ell 2 d result 0.2500 max increase possible 0.3750
k 1 size g 4 size ell 4 d result 0.1250 max increase possible 0.1250
k 2 size g 3 size ell 3 d result 0.0312 max increase possible 0.0469
=== DONE ===
[00000.4516 ms / 0.0005 s / 0.0000 min] blame
[00000.0157 ms / 0.0000 s / 0.0000 min] influence
[00000.0691 ms / 0.0001 s / 0.0000 min] omega
influence 0.25000, 0.40625 <= blame <= 0.40625, blame diff 0.00000


In [3]:
import importance_measures.sharpSAT as ssat
f = ssat.Formula.parse(formula)
x = "x0"
solver = ssat.SharpSAT()
print(f"satcount = {solver.satcount(f)}")
infl = (time_op)(ssat.influence)(f, x, solver)
blame, ub = time_op(ssat.blame)(f, x, solver, rho=lambda x: 2**-x, cutoff=0.01, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

satcount = 14
[01006.4905 ms / 1.0065 s / 0.0168 min] influence
